Die wichtigsten Python-Bibilotheken und/oder Module für die Textandalyse werden importiert. Dazu gehören die Pandas-Bibiliothek (Datenverarbeitung), Numpy (numerische Berechnung), scikit-learn für die Vektorisierung, NLTK für die Sprachverarbeitung (Tokenisierung, Stoppwortentfernung, ...) und Gensim für die Berechung des Coherence-Scores.

In [1]:
import re 
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

Den Inhalt der Datei Comcast.csv auch 'Data Frame' (df) genannt mithilfe von Pandas (pd) einlesen und anschließend den Inhalt der Spalte 'Customer Complaint' mit der Funktion str.lower() in Kleinbuchstaben umwandeln. Abschließend wird die Funktionalität geprüft, indem der Inhalt der ersten fünf Zeilen ausgegeben wird.

In [5]:
# Inhalt der CSV 'Comcast.csv' lesen
df = pd.read_csv ('Comcast.csv')

# Inhalt der Spalte 'Customer Complaint in Kleinbuchstaben umwandeln 
df['Customer Complaint'] = df['Customer Complaint'].str.lower()
print (df['Customer Complaint'].head(5))

0                        comcast cable internet speeds
1         payment disappear - service got disconnected
2                                    speed and service
3    comcast imposed a new usage cap of 300gb that ...
4           comcast not working and no service to boot
Name: Customer Complaint, dtype: object


Als nächstes werden die Wörter der Spalte 'Customer Complaint' aus df tokenisiert. Dafür wird die Funktion word_tokenize aus der Unterbibliothek nltk.tokenize verwendet. Abschließend wird der Inhalt der oberen fünf Zeilen ausgegeben.

In [6]:
# Wörter mithilfe der Funktion word-tokenize aus der Unterbibiliothek nltk.tokenize tokenisiert.
df['Customer Complaint'] = df['Customer Complaint'].apply(word_tokenize)
print (df['Customer Complaint'].head(5))


0                   [comcast, cable, internet, speeds]
1    [payment, disappear, -, service, got, disconne...
2                                [speed, and, service]
3    [comcast, imposed, a, new, usage, cap, of, 300...
4    [comcast, not, working, and, no, service, to, ...
Name: Customer Complaint, dtype: object


Im nächsten Schritt wird eStopWords mit den englischen Stoppwörtern der NLTK-Bibilothek befüllt. Die Liste wird zusätzlich um das Wort 'Comcast' erweitert, da dieses in vielen Beschwerden auftaucht, aber keine relevante Bedeutung hat. Folgend werden die Stoppwörter aus den Data Frame entfernt. Um die Liste der Stoppwörter zu prüfen, wurde eStopWords ausgegeben.

In [23]:
# eStopWords mit den englischen Stoppwörtern füllen
eStopWords = set(stopwords.words('english'))
#Comcast als zusätzliches Stoppwort
eStopWords.add('comcast')
# Stoppwörter entfernen
df['Customer Complaint'] = df['Customer Complaint'].apply(lambda x: [word for word in x if word not in eStopWords])
print ('Übersicht Stoppwörter: \n', eStopWords)
print ('\nÜbersicht nach entfernen der Wörter in df:\n',(df['Customer Complaint'].head(5)))

Übersicht Stoppwörter: 
 {'under', 'some', "you're", "needn't", 'during', 've', 'such', 't', 'y', 'wouldn', 'when', 'his', 'they', 'then', 'so', "she's", 'should', 'who', 'being', 'she', 'ma', 'theirs', "you'd", "won't", 'those', 'because', 'these', 'after', 'above', 'below', 'from', "should've", 'while', 'comcast', 'out', "it's", 'did', 'into', 's', 'had', 'him', "don't", 'will', 're', "doesn't", 'doing', 'have', 'her', 'what', "mustn't", 'been', 'm', 'are', 'is', 'both', 'them', 'before', 'doesn', 'ours', 'myself', 'how', 'yours', 'until', 'of', 'this', 'o', 'own', 'he', 'as', 'with', "mightn't", 'we', "wasn't", 'than', 'few', 'if', 'themselves', 'were', "shan't", "wouldn't", "hasn't", 'himself', 'the', 'just', 'does', 'up', 'aren', 'your', 'an', 'down', "couldn't", 'each', 'again', 'my', 'yourselves', 'can', 'between', 'was', 'only', 'mightn', 'same', 'having', 'now', 'or', 'by', 'll', 'isn', 'shouldn', 'to', "you'll", 'a', 'yourself', 'hers', "isn't", 'off', 'haven', 'itself', 'tha

Folgend werden die restlichen Wörter in die Grundform gebracht. Dafür wird ein WordNetLemmatizer-Objekt erstellt. Abschließend wird die Methode apply() auf die Spalte 'Customer Complaint' angewendet.


In [24]:
# Wörter in die Grundform bringen
lemmatizer = WordNetLemmatizer()
df['Customer Complaint'] = df['Customer Complaint'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
print (df['Customer Complaint'].head(5))

0                             [cable, internet, speed]
1    [payment, disappear, -, service, got, disconne...
2                                     [speed, service]
3    [imposed, new, usage, cap, 300gb, punishes, st...
4                             [working, service, boot]
Name: Customer Complaint, dtype: object


Daraufhin wird aus der Bibiliothek 'sklearn.feature_extraction.text' die Methode CountVectorizer() verwendet und auf die relevante Spalte des Data Frames angewandt. Wodurch die Häufigkeit jedes Wortes ermittelt werden kann. Die Ausgabe zeigt, wie häufig welches Wort im gesamten Dokument vorkommt.

(0, 193) 1 --> (Zeile, WortID) Häufigkeit

In [29]:
# Bag-of-Words-Vektorisierers
bow_vectorizer = CountVectorizer()
X_bow = bow_vectorizer.fit_transform(df['Customer Complaint'].apply(' '.join))
print(X_bow[:5])

  (0, 193)	1
  (0, 630)	1
  (0, 1102)	1
  (1, 847)	1
  (1, 363)	1
  (1, 1059)	1
  (1, 524)	1
  (1, 369)	1
  (2, 1102)	1
  (2, 1059)	1
  (3, 579)	1
  (3, 774)	1
  (3, 1252)	1
  (3, 205)	1
  (3, 34)	1
  (3, 916)	1
  (3, 1118)	1
  (4, 1059)	1
  (4, 1305)	1
  (4, 174)	1


Als Alternative wird der TF-IDF Ansatz angewandt um die Textdaten zu vektorisieren. Dazu wird das TfidfVectorizer-Objekt initialisiert und die Methode fit_transform() aufgerufen.

(0, 11777) 0.6430858133577498 --> (Zeile, WortID) TF-IDF-Werte

In [35]:
# TF-IDF Ansatz
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Customer Complaint'].apply(' '.join))

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['Customer Complaint'].apply(' '.join))
print(X_tfidf[:5])

  (0, 1102)	0.5490388692311106
  (0, 630)	0.38626604277067456
  (0, 193)	0.7411847706717313
  (1, 369)	0.4644737989465081
  (1, 524)	0.52267711428545
  (1, 1059)	0.17305436113878758
  (1, 363)	0.5505296731664239
  (1, 847)	0.4219502237806612
  (2, 1059)	0.5891421002393777
  (2, 1102)	0.808029446075782
  (3, 1118)	0.4131001959415215
  (3, 916)	0.47872384816487784
  (3, 34)	0.36691507235295934
  (3, 205)	0.20829908713237327
  (3, 1252)	0.2901460397392053
  (3, 774)	0.3624883643671149
  (3, 579)	0.4545041106672627
  (4, 174)	0.7627958774943985
  (4, 1305)	0.6005403756988377
  (4, 1059)	0.23977845281227766


Im kommenden Abschnitt wird die Latent Semantic Analysis (LSA) durchgeführt. Dafür wird das  TruncatedSVD-Objekt initialisiert. Zusätzlich wird eine neue Spalte 'LSA Topic' im df erstellt und mit den jeweiligen Ergebnissen befüllt.

- n_components = Anzahlt der zu behaltenden Hauptkomponenten
- algorithm = Gibt den Algorithmus zur Berechnung der Hauptkomponenten an. 
- n_iter = Gibt die Anzahl der Iterationen an
- random_state = Starwert für die Zufallszahlgenerierung 

In [92]:
# LSA
lsa = TruncatedSVD(n_components=3, algorithm='randomized', n_iter=15, random_state=42)
lsa_output = lsa.fit_transform(X)

# Neue Spalte für jede Komponente im Data frame
for i in range(lsa_output.shape[1]):
    df[f'LSA Topic {i}'] = lsa_output[:, i]
print (lsa_output)


[[ 0.43155685 -0.07368233 -0.27758057]
 [ 0.08432855 -0.02346161  0.02302777]
 [ 0.44907221 -0.11084207 -0.05731274]
 ...
 [ 0.07392891 -0.01611489  0.05263301]
 [ 0.03405208 -0.01051365  0.02018472]
 [ 0.21648739 -0.03727326 -0.14197404]]


Nun wird das Latent Dirichlet Allocation (LDA) durchgeführt. Dafür wird die Methode fit_transform auf den Inhalt der TF-IDF Matrix 'X_tfidf' angewandt. Folgend wird, wie bei LSA, eine Spalte im Data Frame 

- n_components = Anzahl der zu modellierenden Themen
- doc_topic_prior = Verteilung im Dokument.
- topic_word_prior = Prior für die Verteilung der Wörter pro Thema

In [93]:
# LDA
lda = LatentDirichletAllocation(n_components=3, doc_topic_prior=0.9, topic_word_prior=0.9)
lda_output = lda.fit_transform(X)
# Neue Spalte für jede Komponente im Data frame
for i in range(lda_output.shape[1]):
    df[f'LDA Topic {i}'] = lda_output[:, i]
print (lda_output)

[[0.35235782 0.21539621 0.43224597]
 [0.52572275 0.24131836 0.23295889]
 [0.23989553 0.26343671 0.49666776]
 ...
 [0.24646709 0.50787919 0.24565372]
 [0.27160839 0.4858537  0.24253791]
 [0.27414465 0.26178462 0.46407074]]


Folglich wird ein Verzeichnis (Dictionary) erstellt, welches jedem einzigartigen Word in der Spalte 'Customer Complaint' einen eindeutigen numerischen Index zuweist. Dafür wird aus der Gensim-Bibliothek die Dictionary Funktion verwendet. Anschließend wird jede Zeile in ein BoW mithilfe der Funktion doc2bow() umgewandelt. Die Ergebnisse werden in der Liste corpus gespeichert.

[(0, 1), (1, 1), (2, 1)] --> [(Wort1 kommt in Zeile 0 einmal vor), (Wort2 kommt in Zeile 0, einmal vor), ...]

In [94]:
# Verzeichnis für die Themen erstellen
dictionary = Dictionary(df['Customer Complaint'])
# Umwandlung in eine vektorisierte Form durch Berechnung des "Frequency counts"
corpus = [dictionary.doc2bow(doc) for doc in df['Customer Complaint']]
print (corpus)

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)], [(2, 1), (8, 1)], [(9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(8, 1), (17, 1), (18, 1)], [(19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)], [(8, 1), (11, 1), (21, 1), (26, 1), (27, 1)], [(8, 1), (9, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(32, 1), (33, 1)], [(34, 1), (35, 1), (36, 1), (37, 1)], [(5, 1), (8, 1), (38, 1), (39, 1)], [(40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)], [(1, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)], [(1, 1), (2, 1)], [(1, 1), (55, 1), (56, 1), (57, 1)], [(1, 1), (58, 1)], [(1, 1), (2, 1), (59, 1)], [(60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)], [(1, 1), (8, 1), (68, 1)], [(8, 1), (9, 1), (41, 1), (69, 1), (70, 1)], [(1, 1), (71, 1), (72, 1)], [(0, 1), (8, 2), (68, 1), (73, 1)], [(2, 1)], [(71, 1), (74, 1), (75, 1)], [(76, 1)], [(77, 1), (78, 1)], [(8, 1), (73, 1)]

Als nächstes werden die Themen aus dem LDA Modell extrahiert. Dafür wird die Anzahl der wichtigsten Wörter pro Thema definitert (n_top_words). Folgend wird mithilfe einer for-Schleife jedes Thema im Modell durchlaufen und diese anhand der Indizes basierend auf ihren Gewichten in den Themen sortiert.

In [95]:
# Themen aus LDA extrahieren
n_top_words = 2 
topics = []
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features = [feature_names[i] for i in top_features_ind]
    topics.append(top_features)
print (topics)

[['billing', 'issue'], ['service', 'complaint'], ['internet', 'data']]


Im nächsten Schritt wird der Coherence Score für die LDA-Modelle ermittelt. Dafür wird ein Objekt mit den Themen, Textdaten und dem Dictionary erstellt und anschließend mit der 'c_v'-Metrik berechnet.

- Je größer der Wert, umso besser können die Themen interpetiert werden.

In [96]:
# Coherence Score
coherence_model_lda = CoherenceModel(topics=topics, texts=df['Customer Complaint'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print ('Coherence Score LDA:', coherence_lda)

Coherence Score LDA: 0.7168726099131598


For-Schleife zur Ausgabe der Besten Wörter pro Thema

In [97]:
#LDA Themen ausgeben
for i, topic in enumerate(topics):
    print(f"Beste Wörter für Thema {i+1}: {', '.join(topic)}")

Beste Wörter für Thema 1: billing, issue
Beste Wörter für Thema 2: service, complaint
Beste Wörter für Thema 3: internet, data


Im nächsten Schritt wird der Coherence Score für die LSA-Modelle ermittelt. Dafür wurden die Top-Wörter extrahiert und Verwendung der 'c_v'-Metrik berechnet.
- Je größer der Wert, umso besser können die Themen interpetiert werden.

In [98]:
# Berechnung des Coherence Scores für LSA mithilfe von c_v measure
topics = [[feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]] for topic in lsa.components_]
coherence_model_lsa = CoherenceModel(topics=topics, texts=df['Customer Complaint'], dictionary=dictionary, coherence='c_v')
coherence_lsa = coherence_model_lsa.get_coherence()
print('Coherence Score LSA: ', coherence_lsa)

Coherence Score LSA:  0.8768361519156876


For-Schleife zur Ausgabe der Besten Wörter pro Thema

In [99]:
#LSA Themen ausgeben
for i, topic in enumerate(topics):
    print(f"Beste Wörter für Thema {i+1}: {', '.join(topic)}")

Beste Wörter für Thema 1: internet, service
Beste Wörter für Thema 2: cap, data
Beste Wörter für Thema 3: billing, issue


Erzeugen der Datei 'Comcast_Ergebnisse.csv', welche die Ergebnisse enthält.

In [42]:
# Datei für die Ausgabe der Ergebnisse erzeugen
df.to_csv('Comcast_Ergebnisse.csv')